In [ ]:
#pip install nltk

In [ ]:
#pip install spacy

In [1]:
#import libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import sklearn


[nltk_data] Downloading package punkt to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.DataFrame(pd.read_csv('Twitter Sentiment/cleaned_text.csv'))

def helper(string):
    temp = []
    for word in string[1:-1].split(','):
        word = word.strip()
        word = word[1:-1]
        if word:
          temp.append(word)  
    return temp
data.filtered_tokens = data.filtered_tokens.apply(helper)

In [3]:
data = data.iloc[25000:75000]
data.shape

(50000, 8)

In [4]:
# One hot encoding
categories = list(set([item for sublist in data.filtered_tokens for item in sublist]))
#print(categories)

vectorizer = CountVectorizer(binary=True, vocabulary=categories)
sentences = [' '.join(words) for words in data['filtered_tokens']]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

# Convert the one-hot encoded data to a DataFrame
encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns = categories)

# use the print function to check if the one hot encoding is correct
#print(encoded_df.iloc[954,:].sum(),sentences[954]) 
print(encoded_df)
#print(categories)


       hrs  cosi  compro  wthell  blondediggity  thingys  situation  doubtful  \
0        0     0       0       0              0        0          0         0   
1        0     0       0       0              0        0          0         0   
2        0     0       0       0              0        0          0         0   
3        0     0       0       0              0        0          0         0   
4        0     0       0       0              0        0          0         0   
...    ...   ...     ...     ...            ...      ...        ...       ...   
49995    0     0       0       0              0        0          0         0   
49996    0     0       0       0              0        0          0         0   
49997    0     0       0       0              0        0          0         0   
49998    0     0       0       0              0        0          0         0   
49999    0     0       0       0              0        0          0         0   

       treatment  write  ..

In [31]:
count = (data['labels'] == 0).sum()
print(count)

25000


In [35]:
data = data.iloc[25000:75000]
encoded_df = encoded_df.iloc[25000:75000]

print(data.shape)
print(encoded_df.shape)

(50000, 8)
(50000, 50661)


In [5]:
# Split the dataset

X_train, X_test, y_train, y_test=train_test_split(encoded_df, data['labels'], stratify=data['labels'], test_size=0.2, random_state=42)

In [9]:
print(y_train)
len(y_train)

73903    1
35174    0
46519    0
34815    0
31895    0
        ..
65542    1
36522    0
48743    0
42799    0
38724    0
Name: labels, Length: 40000, dtype: int64


40000

In [6]:
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

test = SelectKBest(score_func=chi2, k=1000)
fit = test.fit(X_train, y_train)

X_train_selected = test.transform(X_train)

In [7]:
clf = sklearn.linear_model.LogisticRegression(random_state = 42).fit(X_train_selected, y_train)


c:\Users\Josiah Lee\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
X_test_selected = test.transform(X_test)
y_pred = clf.predict(X_test_selected)

In [9]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.7554
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.72      0.75      5000
           1       0.74      0.79      0.76      5000

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000

